# Week 10: Tutorials (MNE Python Toolkit)

Source: [MNE-Python](https://mne.tools/stable/index.html)

# 1. Introduction

These tutorials cover the basic EEG/MEG pipeline for event-related analysis, introduce the [mne.Info](https://mne.tools/stable/generated/mne.Info.html#mne.Info), [events](https://mne.tools/stable/documentation/glossary.html#term-events), and [mne.Annotations](https://mne.tools/stable/generated/mne.Annotations.html#mne.Annotations) data structures, discuss how sensor locations are handled, and introduce some of the configuration options available.

## 1.1 Overview of MEG/EEG analysis with MNE-Python

This tutorial covers the basic EEG/MEG pipeline for event-related analysis: loading data, epoching, averaging, plotting, and estimating cortical activity from sensor data. It introduces the core MNE-Python data structures [Raw](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw), [Epochs](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs), [Evoked](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked), and [SourceEstimate](https://mne.tools/stable/generated/mne.SourceEstimate.html#mne.SourceEstimate), and covers a lot of ground fairly quickly (at the expense of depth). Subsequent tutorials address each of these topics in greater detail.

We begin by importing the necessary Python modules:

In [ ]:
# !pip install mne
# !pip install ipywidgets
# !pip install pyvistaqt
# !pip install nibabel
# !pip install h5io

In [ ]:
# License: BSD-3-Clause
# Copyright the MNE-Python contributors.

import numpy as np
import mne

### Loading data

MNE-Python data structures are based around the FIF file format from Neuromag, but there are reader functions for [a wide variety of other data formats](https://mne.tools/stable/documentation/implementation.html#data-formats). MNE-Python also has interfaces to a variety of [publicly available datasets](https://mne.tools/stable/documentation/datasets.html#datasets), which MNE-Python can download and manage for you.

We’ll start this tutorial by loading one of the example datasets (called “[Sample](https://mne.tools/stable/documentation/datasets.html#sample-dataset)”), which contains EEG and MEG data from one subject performing an audiovisual experiment, along with structural MRI scans for that subject. The [mne.datasets.sample.data_path](https://mne.tools/stable/generated/mne.datasets.sample.data_path.html#mne.datasets.sample.data_path) function will automatically download the dataset if it isn’t found in one of the expected locations, then return the directory path to the dataset (see the documentation of [data_path](https://mne.tools/stable/generated/mne.datasets.sample.data_path.html#mne.datasets.sample.data_path) for a list of places it checks before downloading). Note also that for this tutorial to run smoothly on our servers, we’re using a filtered and downsampled version of the data (`sample_audvis_filt-0-40_raw.fif`), but an unfiltered version (`sample_audvis_raw.fif`) is also included in the sample dataset and could be substituted here when running the tutorial locally.

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = (
    sample_data_folder / "MEG" / "sample" / "sample_audvis_filt-0-40_raw.fif"
)
raw = mne.io.read_raw_fif(sample_data_raw_file)

By default, [read_raw_fif](https://mne.tools/stable/generated/mne.io.read_raw_fif.html#mne.io.read_raw_fif) displays some information about the file it’s loading; for example, here it tells us that there are four “projection items” in the file along with the recorded data; those are [SSP projectors](https://mne.tools/stable/documentation/glossary.html#term-projector) calculated to remove environmental noise from the MEG signals, plus a projector to mean-reference the EEG channels; these are discussed in the tutorial calculated to remove environmental noise from the MEG signals, plus a projector to mean-reference the EEG channels; these are discussed in the tutorial [Background on projectors and projections](https://mne.tools/stable/auto_tutorials/preprocessing/45_projectors_background.html#tut-projectors-background). In addition to the information displayed during loading, you can get a glimpse of the basic details of a [Raw](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw) object by printing it; even more is available by printing its `info` attribute (a [dictionary-like object](https://mne.tools/stable/generated/mne.Info.html#mne.Info) that is preserved across [Raw](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw), [Epochs](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs), and [Evoked](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked) objects). The `info` data structure keeps track of channel locations, applied filters, projectors, etc. Notice especially the `chs` entry, showing that MNE-Python detects different sensor types and handles each appropriately. See [The Info data structure](https://mne.tools/stable/auto_tutorials/intro/30_info.html#tut-info-class) for more on the [Info](https://mne.tools/stable/generated/mne.Info.html#mne.Info) class.

In [ ]:
print(raw)
print(raw.info)

`Raw` objects also have several built-in plotting methods; here we show the power spectral density (PSD) for each sensor type with `compute_psd`, as well as a plot of the raw sensor traces with `plot`. In the PSD plot, we’ll only plot frequencies below 50 Hz (since our data are low-pass filtered at 40 Hz). In interactive Python sessions, `plot` is interactive and allows scrolling, scaling, bad channel marking, annotations, projector toggling, etc.

In [ ]:
raw.compute_psd(fmax=50).plot(picks="data", exclude="bads")
raw.plot(duration=5, n_channels=30)

### Preprocessing

MNE-Python supports a variety of preprocessing approaches and techniques (maxwell filtering, signal-space projection, independent components analysis, filtering, downsampling, etc); see the full list of capabilities in the `mne.preprocessing` and `mne.filter` submodules. Here we’ll clean up our data by performing independent components analysis (`ICA`); for brevity we’ll skip the steps that helped us determined which components best capture the artifacts (see `Repairing artifacts with ICA` for a detailed walk-through of that process).

In [ ]:
# set up and fit the ICA
ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
ica.fit(raw)
ica.exclude = [1, 2]  # details on how we picked these are omitted here
ica.plot_properties(raw, picks=ica.exclude)

Once we’re confident about which component(s) we want to remove, we pass them as the `exclude` parameter and then apply the ICA to the raw signal. The `apply` method requires the raw data to be loaded into memory (by default it’s only read from disk as-needed), so we’ll use `load_data` first. We’ll also make a copy of the `Raw` object so we can compare the signal before and after artifact removal side-by-side:

In [ ]:
orig_raw = raw.copy()
raw.load_data()
ica.apply(raw)

# show some frontal channels to clearly illustrate the artifact removal
chs = [
    "MEG 0111",
    "MEG 0121",
    "MEG 0131",
    "MEG 0211",
    "MEG 0221",
    "MEG 0231",
    "MEG 0311",
    "MEG 0321",
    "MEG 0331",
    "MEG 1511",
    "MEG 1521",
    "MEG 1531",
    "EEG 001",
    "EEG 002",
    "EEG 003",
    "EEG 004",
    "EEG 005",
    "EEG 006",
    "EEG 007",
    "EEG 008",
]
chan_idxs = [raw.ch_names.index(ch) for ch in chs]
orig_raw.plot(order=chan_idxs, start=12, duration=4)
raw.plot(order=chan_idxs, start=12, duration=4)

### Detecting experimental events

The sample dataset includes several `“STIM” channels` that recorded electrical signals sent from the stimulus delivery computer (as brief DC shifts / squarewave pulses). These pulses (often called “triggers”) are used in this dataset to mark experimental events: stimulus onset, stimulus type, and participant response (button press). The individual STIM channels are combined onto a single channel, in such a way that voltage levels on that channel can be unambiguously decoded as a particular event type. On older Neuromag systems (such as that used to record the sample data) this summation channel was called `STI 014`, so we can pass that channel name to the `mne.find_events` function to recover the timing and identity of the stimulus events.

In [ ]:
events = mne.find_events(raw, stim_channel="STI 014")
print(events[:5])  # show the first 5

The resulting events array is an ordinary 3-column `NumPy array`, with sample number in the first column and integer event ID in the last column; the middle column is usually ignored. Rather than keeping track of integer event IDs, we can provide an event dictionary that maps the integer IDs to experimental conditions or events. In this dataset, the mapping looks like this:

Event ID | Condition
-------- | ---------
1        | auditory stimulus (tone) to the left ear
2        | auditory stimulus (tone) to the right ear
3        | visual stimulus (checkerboard) to the left visual field
4        | visual stimulus (checkerboard) to the right visual field
5        | smiley face (catch trial)
32       | subject button press

In [ ]:
event_dict = {
    "auditory/left": 1,
    "auditory/right": 2,
    "visual/left": 3,
    "visual/right": 4,
    "smiley": 5,
    "buttonpress": 32,
}

Event dictionaries like this one are used when extracting epochs from continuous data; the `/` character in the dictionary keys allows pooling across conditions by requesting partial condition descriptors (i.e., requesting `'auditory'` will select all epochs with Event IDs 1 and 2; requesting `'left'` will select all epochs with Event IDs 1 and 3). An example of this is shown in the next section. There is also a convenient `plot_events` function for visualizing the distribution of events across the duration of the recording (to make sure event detection worked as expected). Here we’ll also make use of the `Info` attribute to get the sampling frequency of the recording (so our x-axis will be in seconds instead of in samples).

In [ ]:
fig = mne.viz.plot_events(
    events, event_id=event_dict, sfreq=raw.info["sfreq"], first_samp=raw.first_samp
)

For paradigms that are not event-related (e.g., analysis of resting-state data), you can extract regularly spaced (possibly overlapping) spans of data by creating events using `mne.make_fixed_length_events` and then proceeding with epoching as described in the next section.

### Epoching continuous data

The `Raw` object and the events array are the bare minimum needed to create an `Epochs` object, which we create with the `Epochs` class constructor. Here we’ll also specify some data quality constraints: we’ll reject any epoch where peak-to-peak signal amplitude is beyond reasonable limits for that channel type. This is done with a rejection dictionary; you may include or omit thresholds for any of the channel types present in your data. The values given here are reasonable for this particular dataset, but may need to be adapted for different hardware or recording conditions. For a more automated approach, consider using the `autoreject package`.

In [ ]:
reject_criteria = dict(
    mag=4000e-15,  # 4000 fT
    grad=4000e-13,  # 4000 fT/cm
    eeg=150e-6,  # 150 µV
    eog=250e-6,
)  # 250 µV

We’ll also pass the event dictionary as the `event_id` parameter (so we can work with easy-to-pool event labels instead of the integer event IDs), and specify `tmin` and `tmax` (the time relative to each event at which to start and end each epoch). As mentioned above, by default `Raw` and `Epochs` data aren’t loaded into memory (they’re accessed from disk only when needed), but here we’ll force loading into memory using the `preload=True` parameter so that we can see the results of the rejection criteria being applied:

In [ ]:
epochs = mne.Epochs(
    raw,
    events,
    event_id=event_dict,
    tmin=-0.2,
    tmax=0.5,
    reject=reject_criteria,
    preload=True,
)

Next we’ll pool across left/right stimulus presentations so we can compare auditory versus visual responses. To avoid biasing our signals to the left or right, we’ll use `equalize_event_counts` first to randomly sample epochs from each condition to match the number of epochs present in the condition with the fewest good epochs.

In [ ]:
conds_we_care_about = ["auditory/left", "auditory/right", "visual/left", "visual/right"]
epochs.equalize_event_counts(conds_we_care_about)  # this operates in-place
aud_epochs = epochs["auditory"]
vis_epochs = epochs["visual"]
del raw, epochs  # free up memory

Like `Raw` objects, `Epochs` objects also have a number of built-in plotting methods. One is `plot_image`, which shows each epoch as one row of an image map, with color representing signal magnitude; the average evoked response and the sensor location are shown below the image:

In [ ]:
aud_epochs.plot_image(picks=["MEG 1332", "EEG 021"])

### Time-frequency analysis

The `mne.time_frequency` submodule provides implementations of several algorithms to compute time-frequency representations, power spectral density, and cross-spectral density. Here, for example, we’ll compute for the auditory epochs the induced power at different frequencies and times, using Morlet wavelets. On this dataset the result is not especially informative (it just shows the evoked “auditory N100” response); see [here](https://mne.tools/stable/auto_tutorials/time-freq/20_sensors_time_frequency.html#inter-trial-coherence) for a more extended example on a dataset with richer frequency content.

In [ ]:
frequencies = np.arange(7, 30, 3)
power = mne.time_frequency.tfr_morlet(
    aud_epochs, n_cycles=2, return_itc=False, freqs=frequencies, decim=3
)
power.plot(["MEG 1332"])

### Estimating evoked responses

Now that we have our conditions in `aud_epochs` and `vis_epochs`, we can get an estimate of evoked responses to auditory versus visual stimuli by averaging together the epochs in each condition. This is as simple as calling the `average` method on the `Epochs` object, and then using a function from the `mne.viz` module to compare the global field power for each sensor type of the two `Evoked` objects:

In [ ]:
aud_evoked = aud_epochs.average()
vis_evoked = vis_epochs.average()

mne.viz.plot_compare_evokeds(
    dict(auditory=aud_evoked, visual=vis_evoked),
    legend="upper left",
    show_sensors="upper right",
)

We can also get a more detailed view of each `Evoked` object using other plotting methods such as `plot_joint` or `plot_topomap`. Here we’ll examine just the EEG channels, and see the classic auditory evoked N100-P200 pattern over dorso-frontal electrodes, then plot scalp topographies at some additional arbitrary times:

In [ ]:
aud_evoked.plot_joint(picks="eeg")
aud_evoked.plot_topomap(times=[0.0, 0.08, 0.1, 0.12, 0.2], ch_type="eeg")

Evoked objects can also be combined to show contrasts between conditions, using the `mne.combine_evoked` function. A simple difference can be generated by passing `weights=[1, -1]`. We’ll then plot the difference wave at each sensor using `plot_topo`:

In [ ]:
evoked_diff = mne.combine_evoked([aud_evoked, vis_evoked], weights=[1, -1])
evoked_diff.pick(picks="mag").plot_topo(color="r", legend=False)

### Inverse modeling

Finally, we can estimate the origins of the evoked activity by projecting the sensor data into this subject’s `source space` (a set of points either on the cortical surface or within the cortical volume of that subject, as estimated by structural MRI scans). MNE-Python supports lots of ways of doing this (dynamic statistical parametric mapping, dipole fitting, beamformers, etc.); here we’ll use minimum-norm estimation (MNE) to generate a continuous map of activation constrained to the cortical surface. MNE uses a linear `inverse operator` to project EEG+MEG sensor measurements into the source space. The inverse operator is computed from the `forward solution` for this subject and an estimate of `the covariance of sensor measurements`. For this tutorial we’ll skip those computational steps and load a pre-computed inverse operator from disk (it’s included with the `sample data`). Because this “inverse problem” is underdetermined (there is no unique solution), here we further constrain the solution by providing a regularization parameter specifying the relative smoothness of the current estimates in terms of a signal-to-noise ratio (where “noise” here is akin to baseline activity level across all of cortex).

In [ ]:
# load inverse operator
inverse_operator_file = (
    sample_data_folder / "MEG" / "sample" / "sample_audvis-meg-oct-6-meg-inv.fif"
)
inv_operator = mne.minimum_norm.read_inverse_operator(inverse_operator_file)
# set signal-to-noise ratio (SNR) to compute regularization parameter (λ²)
snr = 3.0
lambda2 = 1.0 / snr**2
# generate the source time course (STC)
stc = mne.minimum_norm.apply_inverse(
    vis_evoked, inv_operator, lambda2=lambda2, method="MNE"
)  # or dSPM, sLORETA, eLORETA

Finally, in order to plot the source estimate on the subject’s cortical surface we’ll also need the path to the sample subject’s structural MRI files (the `subjects_dir`):

In [ ]:
# path to subjects' MRI files
subjects_dir = sample_data_folder / "subjects"
# plot the STC
stc.plot(
    initial_time=0.1, hemi="split", views=["lat", "med"], subjects_dir=subjects_dir
)

## 1.2 Modifying data in-place

Many of MNE-Python’s data objects (`Raw`, `Epochs`, `Evoked`, etc) have methods that modify the data in-place (either optionally or obligatorily). This can be advantageous when working with large datasets because it reduces the amount of computer memory needed to perform the computations. However, it can lead to unexpected results if you’re not aware that it’s happening. This tutorial provides a few examples of in-place processing, and how and when to avoid it.

As usual we’ll start by importing the modules we need and loading some `example data`:

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = sample_data_folder / "MEG" / "sample" / "sample_audvis_raw.fif"
# the preload flag loads the data into memory now
raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)
raw.crop(tmax=10.0)  # raw.crop() always happens in-place

### Signal processing

Most MNE-Python data objects have built-in methods for filtering, including high-, low-, and band-pass filters (`filter`), band-stop filters (`notch_filter`), Hilbert transforms (`apply_hilbert`), and even arbitrary or user-defined functions (`apply_function`). These typically always modify data in-place, so if we want to preserve the unprocessed data for comparison, we must first make a copy of it. For example:

In [ ]:
original_raw = raw.copy()
raw.apply_hilbert()
print(
    f"original data type was {original_raw.get_data().dtype}, after "
    f"apply_hilbert the data type changed to {raw.get_data().dtype}."
)

### Channel picking

Another group of methods where data is modified in-place are the channel-picking methods. For example:

In [ ]:
print(f'original data had {original_raw.info["nchan"]} channels.')
original_raw.pick("eeg")  # selects only the EEG channels
print(f'after picking, it has {original_raw.info["nchan"]} channels.')

Note also that when picking only EEG channels, projectors that affected only the magnetometers were dropped, since there are no longer any magnetometer channels.

### The `copy` parameter

Above we saw an example of using the `copy` method to facilitate comparing data before and after processing. This is not needed when using certain MNE-Python functions, because they have a function parameter where you can specify `copy=True` (return a modified copy of the data) or `copy=False` (operate in-place). For example, `mne.set_eeg_reference` is one such function; notice that here we plot `original_raw` after the rereferencing has been done, but `original_raw` is unaffected because we specified `copy=True`:

In [ ]:
rereferenced_raw, ref_data = mne.set_eeg_reference(original_raw, ["EEG 003"], copy=True)
fig_orig = original_raw.plot()
fig_reref = rereferenced_raw.plot()

Another example is the picking function `mne.pick_info`, which operates on `mne.Info` dictionaries rather than on data objects. See [The Info data structure](https://mne.tools/stable/auto_tutorials/intro/30_info.html#tut-info-class) for details.

## 1.3 Parsing events from raw data

This tutorial describes how to read experimental events from raw recordings, and how to convert between the two different representations of events within MNE-Python (Events arrays and Annotations objects).

In the `introductory tutorial` we saw an
example of reading experimental events from a :term:`"STIM" channel`; here we'll discuss `events` and `annotations` more broadly, give more detailed information about reading from STIM channels, and give an example of reading events that are in a marker file or included in the data file as an embedded array. The tutorials `Working with events` and `Annotation continuous data` discuss how to plot, combine, load, save, and export `events` and `mne.Annotations` (respectively), and the latter tutorial also covers interactive annotation of `mne.io.Raw` objects.

We'll begin by loading the Python modules we need, and loading the same
`example data` we used in the `introductory tutorial`, but to save memory we'll crop the `mne.io.Raw` object
to just 60 seconds before loading it into RAM:

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = sample_data_folder / "MEG" / "sample" / "sample_audvis_raw.fif"
raw = mne.io.read_raw_fif(sample_data_raw_file)
raw.crop(tmax=60).load_data()

### The Events and Annotations data structures

Generally speaking, both the Events and `mne.Annotations` data structures serve the same purpose: they provide a mapping between times during an EEG/MEG recording and a description of what happened at those times. In other words, they associate a *when* with a *what*. The main differences are:

1. **Units**: the Events data structure represents the *when* in terms of
   samples, whereas the `mne.Annotations` data structure represents
   the *when* in seconds.
2. **Limits on the description**: the Events data structure represents the
   *what* as an integer "Event ID" code, whereas the `mne.Annotations` data
   structure represents the *what* as a string.
3. **How duration is encoded**: Events in an Event array do not have a
   duration (though it is possible to represent duration with pairs of
   onset/offset events within an Events array), whereas each element of an
   `mne.Annotations` object necessarily includes a duration (though
   the duration can be zero if an instantaneous event is desired).
4. **Internal representation**: Events are stored as an ordinary `NumPy array`, whereas `mne.Annotations` is
   a `list`-like class defined in MNE-Python.

### What is a STIM channel?

A `stim channel` (short for “stimulus channel”) is a channel that does not receive signals from an EEG, MEG, or other sensor. Instead, STIM channels record voltages (usually short, rectangular DC pulses of fixed magnitudes sent from the experiment-controlling computer) that are time-locked to experimental events, such as the onset of a stimulus or a button-press response by the subject (those pulses are sometimes called `TTL` pulses, event pulses, trigger signals, or just “triggers”). In other cases, these pulses may not be strictly time-locked to an experimental event, but instead may occur in between trials to indicate the type of stimulus (or experimental condition) that is about to occur on the upcoming trial.

The DC pulses may be all on one STIM channel (in which case different experimental events or trial types are encoded as different voltage magnitudes), or they may be spread across several channels, in which case the channel(s) on which the pulse(s) occur can be used to encode different events or conditions. Even on systems with multiple STIM channels, there is often one channel that records a weighted sum of the other STIM channels, in such a way that voltage levels on that channel can be unambiguously decoded as particular event types. On older Neuromag systems (such as that used to record the sample data) this “summation channel” was typically `STI 014`; on newer systems it is more commonly `STI101`. You can see the STIM channels in the raw data file here:

In [ ]:
raw.copy().pick(picks="stim").plot(start=3, duration=6)

You can see that `STI 014` (the summation channel) contains pulses of different magnitudes whereas pulses on other channels have consistent magnitudes. You can also see that every time there is a pulse on one of the other STIM channels, there is a corresponding pulse on `STI 014`.

### Converting a STIM channel signal to an Events array

If your data has events recorded on a STIM channel, you can convert them into an events array using `find_events`. The sample number of the onset (or offset) of each pulse is recorded as the event time, the pulse magnitudes are converted into integers, and these pairs of sample numbers plus integer codes are stored in `NumPy arrays` (usually called “the events array” or just “the events”). In its simplest form, the function requires only the `Raw` object, and the name of the channel(s) from which to read events:

In [ ]:
events = mne.find_events(raw, stim_channel="STI 014")
print(events[:5])  # show the first 5

If you don’t provide the name of a STIM channel, `find_events` will first look for MNE-Python `config variables` for variables `MNE_STIM_CHANNEL`, `MNE_STIM_CHANNEL_1`, etc. If those are not found, channels `STI 014` and `STI101` are tried, followed by the first channel with type “STIM” present in `raw.ch_names`. If you regularly work with data from several different MEG systems with different STIM channel names, setting the `MNE_STIM_CHANNEL` config variable may not be very useful, but for researchers whose data is all from a single system it can be a time-saver to configure that variable once and then forget about it.

`find_events` has several options, including options for aligning events to the onset or offset of the STIM channel pulses, setting the minimum pulse duration, and handling of consecutive pulses (with no return to zero between them). For example, you can effectively encode event duration by passing `output='step'` to `find_events`; see the documentation of `find_events` for details. More information on working with events arrays (including how to plot, combine, load, and save event arrays) can be found in the tutorial `Working with events`.

### Reading embedded events as Annotations

Some EEG/MEG systems generate files where events are stored in a separate data array rather than as pulses on one or more STIM channels. For example, the EEGLAB format stores events as a collection of arrays in the `.set` file. When reading those files, MNE-Python will automatically convert the stored events into an `Annotations` object and store it as the `annotations` attribute of the Raw` object:

In [ ]:
testing_data_folder = mne.datasets.testing.data_path()
eeglab_raw_file = testing_data_folder / "EEGLAB" / "test_raw.set"
eeglab_raw = mne.io.read_raw_eeglab(eeglab_raw_file)
print(eeglab_raw.annotations)

The core data within an `Annotations` object is accessible through three of its attributes: `onset`, `duration`, and `description`. Here we can see that there were 154 events stored in the EEGLAB file, they all had a duration of zero seconds, there were two different types of events, and the first event occurred about 1 second after the recording began:

In [ ]:
print(len(eeglab_raw.annotations))
print(set(eeglab_raw.annotations.duration))
print(set(eeglab_raw.annotations.description))
print(eeglab_raw.annotations.onset[0])

More information on working with `Annotations` objects, including how to add annotations to `Raw` objects interactively, and how to plot, concatenate, load, save, and export `Annotations` objects can be found in the tutorial `Annotating continuous data`.

### Converting between Events arrays and Annotations objects

Once your experimental events are read into MNE-Python (as either an Events array or an `Annotations` object), you can easily convert between the two formats as needed. You might do this because, e.g., an Events array is needed for epoching continuous data, or because you want to take advantage of the “annotation-aware” capability of some functions, which automatically omit spans of data if they overlap with certain annotations.

To convert an `Annotations` object to an Events array, use the function `mne.events_from_annotations` on the `Raw` file containing the annotations. This function will assign an integer Event ID to each unique element of `raw.annotations.description`, and will return the mapping of descriptions to integer Event IDs along with the derived Event array. By default, one event will be created at the onset of each annotation; this can be modified via the chunk_duration parameter of `events_from_annotations` to create equally spaced events within each annotation span (see `Making multiple events per annotation`, below, or see `Making equally-spaced Events arrays` for direct creation of an Events array of equally-spaced events).

In [ ]:
events_from_annot, event_dict = mne.events_from_annotations(eeglab_raw)
print(event_dict)
print(events_from_annot[:5])

If you want to control which integers are mapped to each unique description value, you can pass a `dict` specifying the mapping as the `event_id` parameter of `events_from_annotations`; this `dict` will be returned unmodified as the `event_dict`.

Note that this `event_dict` can be used when creating `Epochs` from `Raw` objects, as demonstrated in the tutorial `The Epochs data structure: discontinuous data`.

In [ ]:
custom_mapping = {"rt": 77, "square": 42}
(events_from_annot, event_dict) = mne.events_from_annotations(
    eeglab_raw, event_id=custom_mapping
)
print(event_dict)
print(events_from_annot[:5])

To make the opposite conversion (from an Events array to an `Annotations` object), you can create a mapping from integer Event ID to string descriptions, use `annotations_from_events` to construct the `Annotations` object, and call the `set_annotations` method to add the annotations to the `Raw` object.

Because the `sample data` was recorded on a Neuromag system (where sample numbering starts when the acquisition system is initiated, not when the recording is initiated), we also need to pass in the `orig_time` parameter so that the onsets are properly aligned relative to the start of recording:

In [ ]:
mapping = {
    1: "auditory/left",
    2: "auditory/right",
    3: "visual/left",
    4: "visual/right",
    5: "smiley",
    32: "buttonpress",
}
annot_from_events = mne.annotations_from_events(
    events=events,
    event_desc=mapping,
    sfreq=raw.info["sfreq"],
    orig_time=raw.info["meas_date"],
)
raw.set_annotations(annot_from_events)

Now, the annotations will appear automatically when plotting the raw data, and will be color-coded by their label value:

In [ ]:
raw.plot(start=5, duration=5)

### Making multiple events per annotation

As mentioned above, you can generate equally-spaced events from an `Annotations` object using the `chunk_duration` parameter of `events_from_annotations`. For example, suppose we have an annotation in our `Raw` object indicating when the subject was in REM sleep, and we want to perform a resting-state analysis on those spans of data. We can create an Events array with a series of equally-spaced events within each “REM” span, and then use those events to generate (potentially overlapping) epochs that we can analyze further.

In [ ]:
# create the REM annotations
rem_annot = mne.Annotations(onset=[5, 41], duration=[16, 11], description=["REM"] * 2)
raw.set_annotations(rem_annot)
(rem_events, rem_event_dict) = mne.events_from_annotations(raw, chunk_duration=1.5)

Now we can check that our events indeed fall in the ranges 5-21 seconds and 41-52 seconds, and are ~1.5 seconds apart (modulo some jitter due to the sampling frequency). Here are the event times rounded to the nearest millisecond:

In [ ]:
print(np.round((rem_events[:, 0] - raw.first_samp) / raw.info["sfreq"], 3))

Other examples of resting-state analysis can be found in the online documentation for [make_fixed_length_events](https://mne.tools/stable/generated/mne.make_fixed_length_events.html#mne.make_fixed_length_events).

## 1.4 The Info data structure

This tutorial describes the `mne.Info` data structure, which keeps track of various recording details, and is attached to `Raw`, `Epochs`, and `Evoked` objects.

We will begin by loading the Python modules we need, and loading the `same example` data we used in the `introductory tutorial`:

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = (
    sample_data_folder / "MEG" / "sample" / "sample_audvis_filt-0-40_raw.fif"
)
raw = mne.io.read_raw_fif(sample_data_raw_file)

As seen in the `introductory tutorial`, when a `Raw` object is loaded, an `Info` object is created automatically, and stored in the `raw.info` attribute:

In [ ]:
print(raw.info)

However, it is not strictly necessary to load the `Raw` object in order to view or edit the `Info` object; you can extract all the relevant information into a stand-alone `Info` object using `mne.io.read_info()`:

In [ ]:
info = mne.io.read_info(sample_data_raw_file)
print(info)

As you can see, the `Info` object keeps track of a lot of information about:

- the recording system (gantry angle, HPI details, sensor digitizations, channel names, …)

- the experiment (project name and ID, subject information, recording date, experimenter name or ID, …)

- the data (sampling frequency, applied filter frequencies, bad channels, projectors, …)

The complete list of fields is given in `the API documentation`.

### Querying the `Info` object

The fields in a `Info` object act like Python `dictionary` keys, using square brackets and strings to access the contents of a field:

In [ ]:
print(info.keys())
print()  # insert a blank line
print(info["ch_names"])

Most of the fields contain `int`, `float`, or `list` data, but the `chs` field bears special mention: it contains a list of dictionaries (one `dict` per channel) containing everything there is to know about a channel other than the data it recorded. Normally it is not necessary to dig into the details of the `chs` field — various MNE-Python functions can extract the information more cleanly than iterating over the list of dicts yourself — but it can be helpful to know what is in there. Here we show the keys for the first channel’s `dict`:

In [ ]:
print(info["chs"][0].keys())

### Obtaining subsets of channels

It is often useful to convert between channel names and the integer indices identifying rows of the data array where those channels’ measurements are stored. The `Info` object is useful for this task; two convenience functions that rely on the `mne.Info` object for picking channels are `mne.pick_channels()` and `mne.pick_types()`. `pick_channels()` minimally takes a list of all channel names and a list of channel names to include; it is also possible to provide an empty list to include and specify which channels to `exclude` instead:

In [ ]:
print(mne.pick_channels(info["ch_names"], include=["MEG 0312", "EEG 005"]))

print(mne.pick_channels(info["ch_names"], include=[], exclude=["MEG 0312", "EEG 005"]))

`pick_types()` works differently, since channel type cannot always be reliably determined from channel name alone. Consequently, `pick_types()` needs an `Info` object instead of just a list of channel names, and has boolean keyword arguments for each channel type. Default behavior is to pick only MEG channels (and MEG reference channels if present) and exclude any channels already marked as “bad” in the `bads` field of the `Info` object. Therefore, to get all and only the EEG channel indices (including the “bad” EEG channels) we must pass `meg=False` and `exclude=[]`:

In [ ]:
print(mne.pick_types(info, meg=False, eeg=True, exclude=[]))

Note that the `meg‍‍‍‍` and `fnirs` parameters of `pick_types()` accept strings as well as boolean values, to allow selecting only magnetom‍‍eter or‍‍ gradiometer channels (via `meg='mag'` or `meg='grad'`) or to pick only oxyhemoglobin or deoxyhemoglobin channels (via `fnirs='hbo'` or `fnirs='hbr'`, respectively).

A third way to pick channels from an `Info` object is to apply `regular expression` matching to the channel names using `mne.pick_channels_regexp()`. Here the `^` represents the beginning of the string and `.` character matches any single character, so both EEG and EOG channels will be selected:

In [ ]:
print(mne.pick_channels_regexp(info["ch_names"], "^E.G"))

`pick_channels_regexp()` can be especially useful for channels named according to the `10-20` system (e.g., to select all channels ending in “z” to get the midline, or all channels beginning with “O” to get the occipital channels). Note that `pick_channels_regexp()` uses the Python standard module `re` to perform regular expression matching; see the documentation of the `re` module for implementation details.

### Obtaining channel type information

Sometimes it can be useful to know channel type based on its index in the data array. For this case, use `mne.channel_type()`, which takes an `Info` object and a single integer channel index:

In [ ]:
print(mne.channel_type(info, 25))

To obtain several channel types at once, you could embed `channel_type()` in a `list comprehension`, or use the `get_channel_types()` method of a `Raw`, `Epochs`, or `Evoked` instance:

In [ ]:
picks = (25, 76, 77, 319)
print([mne.channel_type(info, x) for x in picks])
print(raw.get_channel_types(picks=picks))

Alternatively, you can get the indices of all channels of all channel types present in the data, using `channel_indices_by_type()`, which returns a `dict` with channel types as keys, and lists of channel indices as values:

In [ ]:
ch_idx_by_type = mne.channel_indices_by_type(info)
print(ch_idx_by_type.keys())
print(ch_idx_by_type["eog"])

### Dropping channels from an Info object

If you want to modify an `Info` object by eliminating some of the channels in it, you can use the `mne.pick_info()` function to pick the channels you want to keep and omit the rest:

In [ ]:
print(info["nchan"])
eeg_indices = mne.pick_types(info, meg=False, eeg=True)
print(mne.pick_info(info, eeg_indices)["nchan"])

In [ ]:
We can also get a nice HTML representation in IPython like this:

In [ ]:
info

By default, `pick_info()` will make a copy of the original `Info` object before modifying it; if you want to modify it in-place, include the parameter `copy=False`.

## 1.5 Working with sensor locations

This tutorial describes how to read and plot sensor locations, and how MNE-Python handles physical locations of sensors. As usual we’ll start by importing the modules we need:

In [ ]:
import mne
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

### About montages and layouts

`Montages` contain sensor positions in 3D (x, y, z in meters), which can be assigned to existing EEG/MEG data. By specifying the locations of sensors relative to the brain, `Montages` play an important role in computing the forward solution and inverse estimates.

In contrast, `Layouts` are idealized 2D representations of sensor positions. They are primarily used for arranging individual sensor subplots in a topoplot or for showing the approximate relative arrangement of sensors as seen from above.

**Note:**
```
If you’re working with EEG data exclusively, you’ll want to use Montages, not layouts. Idealized montages (e.g., those provided by the manufacturer, or the ones shipping with MNE-Python mentioned below) are typically referred to as template montages.
```

### Working with built-in montages

The 3D coordinates of MEG sensors are included in the raw recordings from MEG systems. They are automatically stored in the `info` attribute of the `Raw` object upon loading. EEG electrode locations are much more variable because of differences in head shape. Idealized montages (”`template montages`”) for many EEG systems are included in MNE-Python, and you can get an overview of them by using `mne.channels.get_builtin_montages()`:

In [ ]:
builtin_montages = mne.channels.get_builtin_montages(descriptions=True)
for montage_name, montage_description in builtin_montages:
    print(f"{montage_name}: {montage_description}")

These built-in EEG montages can be loaded with `mne.channels.make_standard_montage`:

In [ ]:
easycap_montage = mne.channels.make_standard_montage("easycap-M1")
print(easycap_montage)

`Montage` objects have a `plot` method for visualizing the sensor locations in 2D or 3D:

In [ ]:
easycap_montage.plot()  # 2D
fig = easycap_montage.plot(kind="3d", show=False)  # 3D
fig = fig.gca().view_init(azim=70, elev=15)  # set view angle for tutorial

Once loaded, a montage can be applied to data with the `set_montage` method, for example `raw.set_montage()`, `epochs.set_montage()`, or `evoked.set_montage()`. This will only work with data whose EEG channel names correspond to those in the montage. (Therefore, we’re loading some EEG data below, and not the usual MNE “sample” dataset.)

You can then visualize the sensor locations via the `plot_sensors()` method.

It is also possible to skip the manual montage loading step by passing the montage name directly to the `set_montage()` method.

In [ ]:
ssvep_folder = mne.datasets.ssvep.data_path()
ssvep_data_raw_path = (
    ssvep_folder / "sub-02" / "ses-01" / "eeg" / "sub-02_ses-01_task-ssvep_eeg.vhdr"
)
ssvep_raw = mne.io.read_raw_brainvision(ssvep_data_raw_path, verbose=False)

# Use the preloaded montage
ssvep_raw.set_montage(easycap_montage)
fig = ssvep_raw.plot_sensors(show_names=True)

# Apply a template montage directly, without preloading
ssvep_raw.set_montage("easycap-M1")
fig = ssvep_raw.plot_sensors(show_names=True)

**Note:**

```
You may have noticed that the figures created via plot_sensors() contain fewer sensors than the result of easycap_montage.plot(). This is because the montage contains all channels defined for that EEG system; but not all recordings will necessarily use all possible channels. Thus when applying a montage to an actual EEG dataset, information about sensors that are not actually present in the data is removed.
```

### Plotting 2D sensor locations like EEGLAB

In MNE-Python, by default the head center is calculated using `fiducial points`. This means that the head circle represents the head circumference at the nasion and ear level, and not where it is commonly measured in the 10–20 EEG system (i.e., above the nasion at T4/T8, T3/T7, Oz, and Fpz).

If you prefer to draw the head circle using 10–20 conventions (which are also used by EEGLAB), you can pass `sphere='eeglab'`:

In [ ]:
fig = ssvep_raw.plot_sensors(show_names=True, sphere="eeglab")

Because the data we’re using here doesn’t contain an Fpz channel, its putative location was approximated automatically.

### Manually controlling 2D channel projection

Channel positions in 2D space are obtained by projecting their actual 3D positions onto a sphere, then projecting the sphere onto a plane. By default, a sphere with origin at `(0, 0, 0)` (x, y, z coordinates) and radius of `0.095` meters (9.5 cm) is used. You can use a different sphere radius by passing a single value as the sphere argument in any function that plots channels in 2D (like `plot` that we use here, but also for example `mne.viz.plot_topomap`):

In [ ]:
fig1 = easycap_montage.plot()  # default radius of 0.095
fig2 = easycap_montage.plot(sphere=0.07)

To change not only the radius, but also the sphere origin, pass a `(x, y, z, radius)` tuple as the `sphere` argument:

In [ ]:
fig = easycap_montage.plot(sphere=(0.03, 0.02, 0.01, 0.075))

### Reading sensor digitization files

In the sample data, the sensor positions are already available in the `info` attribute of the `Raw` object (see the documentation of the reading functions and `set_montage()` for details on how that works). Therefore, we can plot sensor locations directly from the `Raw` object using `plot_sensors()`, which provides similar functionality to `montage.plot()`. In addition, `plot_sensors()` supports channel selection by type, color-coding channels in various ways (by default, channels listed in `raw.info['bads']` will be plotted in red), and drawing in an existing Matplotlib `Axes` object (so the channel positions can easily be added as a subplot in a multi-panel figure):

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_path = sample_data_folder / "MEG" / "sample" / "sample_audvis_raw.fif"
sample_raw = mne.io.read_raw_fif(sample_data_raw_path, preload=False, verbose=False)

fig = plt.figure()
ax2d = fig.add_subplot(121)
ax3d = fig.add_subplot(122, projection="3d")
sample_raw.plot_sensors(ch_type="eeg", axes=ax2d)
sample_raw.plot_sensors(ch_type="eeg", axes=ax3d, kind="3d")
ax3d.view_init(azim=70, elev=15)

The previous 2D topomap reveals irregularities in the EEG sensor positions in the `sample dataset` — this is because the sensor positions in that dataset are digitizations of actual sensor positions on the head rather than idealized sensor positions based on a spherical head model. Depending on the digitization device (e.g., a Polhemus Fastrak digitizer), you need to use different montage reading functions (see `Supported formats for digitized 3D locations`). The resulting `montage` can then be added to `Raw` objects by passing it as an argument to the `set_montage()` method (just as we did before with the name of the predefined `'standard_1020'` montage). Once loaded, locations can be plotted with the `plot()` method and saved with the `save()` method of the `montage` object.

### Visualizing sensors in 3D surface renderings

It is also possible to render an image of an MEG sensor helmet using 3D surface rendering instead of matplotlib. This works by calling `mne.viz.plot_alignment()`:

In [ ]:
fig = mne.viz.plot_alignment(
    sample_raw.info,
    dig=False,
    eeg=False,
    surfaces=[],
    meg=["helmet", "sensors"],
    coord_frame="meg",
)
mne.viz.set_3d_view(fig, azimuth=50, elevation=90, distance=0.5)

Note that `plot_alignment()` requires an `Info` object, and can also render MRI surfaces of the scalp, skull, and brain (by passing a dict with keys like `'head'`, `'outer_skull'` or `'brain'` to the surfaces parameter). This makes the function useful for `assessing coordinate frame transformations`. For examples of various uses of `plot_alignment()`, see `Plotting sensor layouts of EEG systems, Plotting EEG sensors on the scalp`, and `Plotting sensor layouts of MEG systems`.

### Working with layout files

Similar to montages, many layout files are included with MNE-Python. They are stored in the `mne/channels/data/layouts` folder:

In [ ]:
layout_dir = Path(mne.__file__).parent / "channels" / "data" / "layouts"
layouts = sorted(path.name for path in layout_dir.iterdir())
print("\n" "BUILT-IN LAYOUTS\n" "================")
print("\n".join(layouts))

To load a layout file, use the `mne.channels.read_layout` function. You can then visualize the layout using its `plot` method:

In [ ]:
biosemi_layout = mne.channels.read_layout("biosemi")

Similar to the `picks` argument for selecting channels from `Raw` objects, the `plot()` method of `Layout` objects also has a `picks` argument. However, because layouts only contain information about sensor name and location (not sensor type), the `plot()` method only supports picking channels by index (not by name or by type). In the following example, we find the desired indices using `numpy.where()`; selection by name or type is possible with `mne.pick_channels()` or `mne.pick_types()`.

In [ ]:
midline = np.where([name.endswith("z") for name in biosemi_layout.names])[0]
biosemi_layout.plot(picks=midline)

If you have a `Raw` object that contains sensor positions, you can create a `Layout` object with either `mne.channels.make_eeg_layout()` or `mne.channels.find_layout()`.

In [ ]:
layout_from_raw = mne.channels.make_eeg_layout(sample_raw.info)
# same result as mne.channels.find_layout(raw.info, ch_type='eeg')
layout_from_raw.plot()

**Note:**

```
There is no corresponding make_meg_layout() function because sensor locations are fixed in an MEG system (unlike in EEG, where sensor caps deform to fit snugly on a specific head). Therefore, MEG layouts are consistent (constant) for a given system and you can simply load them with mne.channels.read_layout() or use mne.channels.find_layout() with the ch_type parameter (as previously demonstrated for EEG).
```

All `Layout` objects have a `save` method that writes layouts to disk as either `.lout` or `.lay` formats (inferred from the file extension contained in the `fname` argument). The choice between `.lout` and `.lay` format only matters if you need to load the layout file in some other application (MNE-Python can read both formats).